In [ ]:
"""
To write the data to NullModelParameters for networks
That we already know their avaerage number of edges
"""

In [1]:
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
dj.config["display.limit"] = 20

from importlib import reload
import graph_generating_functions_library as g_gen
g_gen= reload(g_gen)

Connecting celiib@10.28.0.34:3306


In [2]:
from graph_generating_functions_library import *

total_graphs = [
"erdos_renyi_random_location",
"watts_strogatz_graph_smallworld_random_location",
"random_tree_random_location",
"random_uniform",
"random_power_law",
"linear_preferencial_attachment_random",
"linear_preferncial_attachment_wheel",
"vertex_duplication",
"vertex_duplication_with_mutation",
"vertex_duplication_with_complement"
]

n_options = np.arange(40,521,40).astype("int")
m_options = np.arange(5,20,5).astype("int")
p_options = np.round(np.arange(0.1,0.9,0.2),2)
p2_options = np.round(np.arange(0.1,0.9,0.2),2)
alpha_options = np.round(np.arange(2.2,3.5,0.1),2)

graph_parameters_to_iterate = {
    erdos_renyi_random_location:dict(
                                    n=n_options,
                                    p=p_options),
    watts_strogatz_graph_smallworld_random_location: dict(
        n=n_options,
        p=p_options,
        m=m_options
    ),
    random_tree_random_location: dict(
        n=n_options),
    
    random_uniform: dict(n=n_options,
                  m=m_options),
    
    random_power_law:dict(n=n_options,
                     alpha=alpha_options),
    
    linear_preferencial_attachment_random:dict(
    n=n_options,
    m=m_options,
        p=p_options,
    ), 
    
    linear_preferncial_attachment_wheel:dict(
    n=n_options,
        m=m_options),
    
    vertex_duplication:dict(n=n_options,
                           p=p_options),
    
    vertex_duplication_with_mutation:dict(n=n_options,
                                         p=p_options,
                                         p2=p2_options),
    
    
    vertex_duplication_with_complement:dict(n=n_options,
                                           p=p_options,
                                           p2=p2_options)
        
}



In [3]:
import numpy as np
from scipy.special import comb
def choose(n,k):
    return comb(n, k, exact=True, repetition=False)

In [4]:
def erdos_renyi_random_location_edge(n,p):
    return choose(n,2)*p,-1

def watts_strogatz_graph_smallworld_random_location_edge(n,p,m):
    return n*m,0

def random_tree_random_location_edge(n):
    return n-1,0

def random_uniform_edge(n,m):
    return n*m,0

def random_power_law_edge(n,alpha,xmin=3):
    """
    Gets the expected number of edges for power law
    """
    
    return np.round(n*xmin*(alpha-1)/(2*(alpha-2)),3),-1

graph_expected_edges_function = {
    erdos_renyi_random_location: erdos_renyi_random_location_edge, 
    watts_strogatz_graph_smallworld_random_location: 
        watts_strogatz_graph_smallworld_random_location_edge,
    
    random_tree_random_location:
        random_tree_random_location_edge,
    
    random_uniform: random_uniform_edge,
    
    random_power_law:random_power_law_edge
        
}

In [5]:
import hashlib

def hash_string(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**8

def get_graph_hash(func_name,param_dict):
    total_s = func_name  
    for k,v in param_dict.items():
        total_s += f"_{k}_{v}"
    #print("total_s = " + total_s)
    return hash_string(total_s)

play_dict = dict(n=10,p=5)
get_graph_hash("Brendan",play_dict)

80771626

In [6]:
graph_models_to_run = [
    "erdos_renyi",
    "small_world",
    "random_tree",
    "random_uniform",
    "power_law"
]

In [7]:
@schema
class NullModelParameters(dj.Manual):
    definition="""
    graph_hash: int unsigned # the hash of the type of graph
    ---
    graph_name : char(60)              # the type of graph used to generate
    n =NULL:          int unsigned      # number of nodes
    p=NULL :          double            # probability of connection ro duplication or rearranging an edge
    m=NULL :          int unsigned      # number of desired edges
    alpha=NULL:       double            # the exponent for the power sequence
    p2=NULL:          double            # another probability that is used for generation
    edge_average  :    double #the average number of edges
    edge_deviation :   double #the deviation of the edge sizes
    """


In [ ]:

from tqdm import tqdm
import itertools


for g_func,g_param_dict in graph_parameters_to_iterate.items():
    print(f"\nWorking on graph:{g_func.__name__}")
    
    if g_func.__name__ not in graph_models_to_run:
        print("Not in list to run so skipping")
        continue
    
    edge_function = graph_expected_edges_function[g_func]
    iterable_loops = [dict([(k,v)]) for k,v in g_param_dict.items()]
    #print("iterable_loops = " + str(iterable_loops))
    parameter_names = [list(k.keys())[0] for k in iterable_loops]

    # initializing list of list  
    all_list = [k[list(k.keys())[0]] for k in iterable_loops]


    parameter_combinations = list(itertools.product(*all_list)) 
    print("len(parameter_combinations) = " + str(len(parameter_combinations)))
    param_dict_combinations = [dict([(k,v) for k,v in zip(parameter_names,j)]) for j in parameter_combinations]
    dicts_to_insert = []
    for u,param_dict in enumerate(tqdm(param_dict_combinations)):
        #calculate the estimated number of edges
        edge_average, edge_deviation = edge_function(**param_dict)
                
        """
        graph_name : char(60)              # the type of graph used to generate
        n =NULL:          int unsigned      # number of nodes
        p=NULL :          double            # probability of connection ro duplication or rearranging an edge
        m=NULL :          int unsigned      # number of desired edges
        alpha=NULL:       double            # the exponent for the power sequence
        p2=NULL:          double            # another probability that is used for generation
        edge_average  :    double #the average number of edges
        edge_deviation :   double #the deviation of the edge sizes
        """
        
        graph_hash = get_graph_hash(g_func.__name__,param_dict)
        
        #compute the edge statistics
        insert_dict = dict(param_dict,
                           graph_hash = graph_hash,
                           graph_name=g_func.__name__,
                           edge_average=edge_average,
                           edge_deviation=edge_deviation
                          )
        #print("dicts_to_insert = " + str(dicts_to_insert))
        dicts_to_insert.append(insert_dict)
        
    m65.NullModelParameters.insert(dicts_to_insert,skip_duplicates=True)
        
    
          

# Running the Erdos Renyi Models that were missing after getting the groups for RandomNetworkStatsTest

In [8]:
"""
Combinations left to run for erdos-renyi:
"""
comb_left = np.array([(40, 153.6),
 (80, 305.2),
 (120, 573.6),
 (160, 770.8),
 (200, 980.4000000000001),
 (240, 1091.6),
 (280, 1390.8),
 (320, 1391.2),
 (360, 1516.4),
 (400, 1618.0),
 (440, 1814.8000000000002),
 (480, 1913.6),
 (520, 2132.0)])

erdos_n = comb_left[:,0]

def erdos_renyi_random_location_edge(n,p):
    return choose(n,2)*p,-1

#the necessary probability
erdos_p = [z[1]/choose(z[0],2) for z in comb_left]

for n,p in zip(erdos_n,erdos_p):
    print(erdos_renyi_random_location_edge(n,p))

(153.6, -1)
(305.2, -1)
(573.6, -1)
(770.8, -1)
(980.4000000000001, -1)
(1091.6, -1)
(1390.8, -1)
(1391.2, -1)
(1516.4, -1)
(1618.0, -1)
(1814.8, -1)
(1913.6, -1)
(2132.0, -1)


In [9]:
# Now write these erdos renyi models to the table

from tqdm import tqdm
import itertools


g_func = erdos_renyi_random_location
edge_function = erdos_renyi_random_location_edge

param_dict_combinations = [dict(n=n,p=p) for n,p in zip(erdos_n,erdos_p)]
dicts_to_insert = []
for u,param_dict in enumerate(tqdm(param_dict_combinations)):
    #calculate the estimated number of edges
    edge_average, edge_deviation = edge_function(**param_dict)

    """
    graph_name : char(60)              # the type of graph used to generate
    n =NULL:          int unsigned      # number of nodes
    p=NULL :          double            # probability of connection ro duplication or rearranging an edge
    m=NULL :          int unsigned      # number of desired edges
    alpha=NULL:       double            # the exponent for the power sequence
    p2=NULL:          double            # another probability that is used for generation
    edge_average  :    double #the average number of edges
    edge_deviation :   double #the deviation of the edge sizes
    """

    graph_hash = get_graph_hash(g_func.__name__,param_dict)


    #compute the edge statistics
    insert_dict = dict(param_dict,
                       graph_hash = graph_hash,
                       graph_name=g_func.__name__,
                       edge_average=edge_average,
                       edge_deviation=edge_deviation
                      )
    #print("dicts_to_insert = " + str(dicts_to_insert))
    dicts_to_insert.append(insert_dict)

m65.NullModelParameters.insert(dicts_to_insert,skip_duplicates=True)
        
    
          

100%|██████████| 13/13 [00:00<00:00, 10019.47it/s]


In [11]:
NullModelParameters() & "graph_name='erdos_renyi'"

graph_hash the hash of the type of graph,graph_name the type of graph used to generate,n number of nodes,p probability of connection ro duplication or rearranging an edge,m number of desired edges,alpha the exponent for the power sequence,p2 another probability that is used for generation,edge_average the average number of edges,edge_deviation the deviation of the edge sizes
5459142,erdos_renyi,440,0.5,None,nan,nan,48290.0,-1.0
7586545,erdos_renyi,360,0.3,None,nan,nan,19386.0,-1.0
7877264,erdos_renyi,440,0.3,None,nan,nan,28974.0,-1.0
9955134,erdos_renyi,320,0.1,None,nan,nan,5104.0,-1.0
10830817,erdos_renyi,360,0.7,None,nan,nan,45234.0,-1.0
12035170,erdos_renyi,480,0.1,None,nan,nan,11496.0,-1.0
12464410,erdos_renyi,160,0.1,None,nan,nan,1272.0,-1.0
12545547,erdos_renyi,400,0.7,None,nan,nan,55860.0,-1.0
13166703,erdos_renyi,160,0.5,None,nan,nan,6360.0,-1.0
13666465,erdos_renyi,280,0.03560675883256528,None,nan,nan,1390.8,-1.0


In [12]:
erdos_p

[0.19692307692307692,
 0.09658227848101265,
 0.08033613445378152,
 0.060597484276729556,
 0.04926633165829146,
 0.03806136680613668,
 0.03560675883256528,
 0.027257053291536052,
 0.02346641906530486,
 0.020275689223057645,
 0.018790639884033962,
 0.016645789839944327,
 0.01579961464354528]

In [ ]:
"""
Erdos_Renyi P has to be a lot smaller

"""